# Physics-Informed Neural Network (PINN) vs Normal Neural Network
## Simple Harmonic Motion (SHM)

We solve the SHM differential equation:
$$ \frac{d^2y}{dt^2} + \omega^2 y = 0 $$

True solution:
$$ y(t) = \sin(\omega t) $$

We compare:
1. PINN (Physics + few data points)
2. Normal Neural Network (data only)

We also compare activation functions:
- Tanh
- Sigmoid

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
omega = 1.0

## Generate Training Data

In [ ]:
t = torch.linspace(0, 10, 200).view(-1,1).to(device)
t.requires_grad = True
y_true = torch.sin(omega * t)

# Few data points (3 anchor points)
t_data = torch.tensor([[0.0],[3.14],[6.28]], device=device)
y_data = torch.sin(omega * t_data)

## Define Neural Network

In [ ]:
class Net(nn.Module):
    def __init__(self, activation='tanh'):
        super().__init__()
        act = nn.Tanh() if activation=='tanh' else nn.Sigmoid()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            act,
            nn.Linear(64, 64),
            act,
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

## Train PINN

In [ ]:
def train_pinn(activation='tanh'):
    model = Net(activation).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(3000):
        optimizer.zero_grad()
        y_pred = model(t)

        dy = torch.autograd.grad(y_pred, t, torch.ones_like(y_pred), create_graph=True)[0]
        d2y = torch.autograd.grad(dy, t, torch.ones_like(dy), create_graph=True)[0]

        physics_loss = torch.mean((d2y + omega**2 * y_pred)**2)
        data_loss = torch.mean((model(t_data) - y_data)**2)

        loss = physics_loss + data_loss
        loss.backward()
        optimizer.step()

    return model

## Train Normal NN (Data Only)

In [ ]:
def train_nn(activation='tanh'):
    model = Net(activation).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(3000):
        optimizer.zero_grad()
        loss = torch.mean((model(t_data) - y_data)**2)
        loss.backward()
        optimizer.step()

    return model

## Train Models

In [ ]:
pinn_tanh = train_pinn('tanh')
nn_tanh = train_nn('tanh')

## Plot PINN Result

In [ ]:
with torch.no_grad():
    plt.figure()
    plt.plot(t.cpu(), y_true.cpu(), label='Actual')
    plt.plot(t.cpu(), pinn_tanh(t).cpu(), '--', label='PINN')
    plt.legend()
    plt.xlabel('t')
    plt.ylabel('y')
    plt.title('PINN (Tanh Activation)')
    plt.show()

## Plot Normal NN Result

In [ ]:
with torch.no_grad():
    plt.figure()
    plt.plot(t.cpu(), y_true.cpu(), label='Actual')
    plt.plot(t.cpu(), nn_tanh(t).cpu(), '--', label='Standard NN')
    plt.legend()
    plt.xlabel('t')
    plt.ylabel('y')
    plt.title('Normal NN (Tanh Activation)')
    plt.show()

## Observations
- PINN follows oscillatory physics even with very few data points.
- Standard NN overfits anchor points and fails to generalize.
- Tanh typically performs better than Sigmoid due to zero-centered gradients.
- Physics constraint dramatically improves extrapolation.